In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

# Load Dataset
# Replace 'dataset.csv' with your dataset file path
df = pd.read_csv('../data/physionet_wo_missing.csv', index_col=0)

# Convert dataset to numpy array (assuming all features are numeric)
data = df.to_numpy()

# Split Dataset
# Specify the test size (e.g., 20% for testing)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Normalize Data
# Use StandardScaler to normalize the features
scaler = StandardScaler()
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

# Convert to Torch Tensors
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_tensor = torch.tensor(train_data, dtype=torch.float32).to(device)
test_tensor = torch.tensor(test_data, dtype=torch.float32).to(device)

# Verify Shapes
print(f"Train Tensor Shape: {train_tensor.shape}")
print(f"Test Tensor Shape: {test_tensor.shape}")
print(f"Device Used: {device}")

Train Tensor Shape: torch.Size([1278, 39])
Test Tensor Shape: torch.Size([320, 39])
Device Used: cuda


In [2]:
import torch
import torch.nn as nn

class TabularTransformer(nn.Module):
    def __init__(self, num_features, d_model=64, num_heads=4, num_layers=2, dropout=0.1):
        super(TabularTransformer, self).__init__()
        
        # 1) Column Embedding Layer
        self.embedding = nn.Linear(1, d_model)  # Embed each feature (column) into d_model dimensions
        
        # 2) Positional Encoding (Column Embedding)
        self.column_embedding = nn.Embedding(num_features, d_model)
        
        # 3) Transformer Encoder Layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=num_heads, 
            dim_feedforward=128, 
            dropout=dropout,
            batch_first=True  # Use batch-first format for better performance
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        
        # 4) Output Layer
        self.output_layer = nn.Linear(d_model, 1)  # Predict one value per feature

    def forward(self, x, column_indices, mask=None):
        """
        x: [batch_size, num_features]
        column_indices: Tensor of column indices for embedding (shape: [num_features])
        mask: Optional mask for self-attention (shape: [batch_size, num_features, num_features])
        """
        # Reshape x to [batch_size, num_features, 1] for embedding
        x = x.unsqueeze(-1)
        
        # Column Embedding
        x_embed = self.embedding(x)
        
        # Add Positional (Column) Embedding
        column_embed = self.column_embedding(column_indices)
        x_embed += column_embed.unsqueeze(0)  # Broadcast across batch
        
        # Pass through Transformer Encoder
        x_encoded = self.transformer_encoder(x_embed, mask=mask)
        
        # Final Output Layer (Predict missing values)
        output = self.output_layer(x_encoded)
        return output.squeeze(-1)  # [batch_size, num_features]

# Model Initialization
num_features = train_tensor.shape[1]  # Number of features (columns in dataset)
model = TabularTransformer(num_features=num_features, d_model=64, num_heads=4, num_layers=2)

# Move model to GPU if available
model = model.to(device)

# Define Column Indices
column_indices = torch.arange(num_features).to(device)

# Example Forward Pass
output = model(train_tensor, column_indices)
print(f"Output shape: {output.shape}")  # Should match [batch_size, num_features]

Output shape: torch.Size([1278, 39])


c:\Users\emith\anaconda3\Lib\site-packages\torch\nn\functional.py:5560: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


In [3]:
def create_missing_mask(data, missing_fraction=0.2):
    """
    Creates a binary mask for missing values.

    Args:
        data (Tensor): Input data (shape: [batch_size, num_features]).
        missing_fraction (float): Fraction of values to mask.

    Returns:
        Tensor: Binary mask of the same shape as `data` (1 = missing, 0 = observed).
    """
    mask = torch.rand(data.shape).to(data.device) < missing_fraction
    return mask.int()

# Example usage
missing_fraction = 0.2  # Mask 20% of the data
mask = create_missing_mask(train_tensor, missing_fraction)
print(f"Mask Shape: {mask.shape}")

Mask Shape: torch.Size([1278, 39])


In [4]:
def compute_loss(predictions, ground_truth, mask):
    """
    Computes the loss only on masked positions.

    Args:
        predictions (Tensor): Model predictions (shape: [batch_size, num_features]).
        ground_truth (Tensor): Original data (shape: [batch_size, num_features]).
        mask (Tensor): Binary mask (1 = missing, 0 = observed).

    Returns:
        Tensor: Scalar loss value.
    """
    mse_loss = nn.MSELoss(reduction='none')
    loss = mse_loss(predictions, ground_truth)
    masked_loss = (loss * mask).sum() / mask.sum()  # Normalize by number of masked positions
    return masked_loss

In [5]:
# Define Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Generate masks and replace masked values with 0
    mask = create_missing_mask(train_tensor, missing_fraction)
    input_with_mask = train_tensor.clone()
    input_with_mask[mask == 1] = 0  # Replace masked positions with 0
    
    # Forward Pass
    predictions = model(input_with_mask, column_indices)
    
    # Compute Loss
    loss = compute_loss(predictions, train_tensor, mask)
    
    # Backward Pass and Optimization
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")


Epoch 1/10, Loss: 1.2442
Epoch 2/10, Loss: 1.3574
Epoch 3/10, Loss: 1.1146
Epoch 4/10, Loss: 1.0673
Epoch 5/10, Loss: 1.0662
Epoch 6/10, Loss: 1.0245
Epoch 7/10, Loss: 1.0768
Epoch 8/10, Loss: 0.9893
Epoch 9/10, Loss: 0.9766
Epoch 10/10, Loss: 1.0551


In [6]:
def evaluate_model(model, test_data, column_indices, missing_fraction=0.2):
    model.eval()
    
    # Generate masks and replace masked values with 0
    mask = create_missing_mask(test_data, missing_fraction)
    input_with_mask = test_data.clone()
    input_with_mask[mask == 1] = 0  # Replace masked positions with 0
    
    # Forward Pass
    with torch.no_grad():
        predictions = model(input_with_mask, column_indices)
    
    # Compute Loss
    loss = compute_loss(predictions, test_data, mask)
    print(f"Test Loss: {loss.item():.4f}")
    return loss

# Evaluate the Model
evaluate_model(model, test_tensor, column_indices)


Test Loss: 1.1588


tensor(1.1588, device='cuda:0')

In [7]:
import torch

def compute_nrmse(predictions, ground_truth, mask):
    """
    Computes the Normalized Root Mean Squared Error (NRMSE) for imputed values.

    Args:
        predictions (Tensor): Model predictions (shape: [batch_size, num_features]).
        ground_truth (Tensor): Original data (shape: [batch_size, num_features]).
        mask (Tensor): Binary mask (1 = missing, 0 = observed).

    Returns:
        float: NRMSE value for the masked positions.
    """
    # Extract only masked values
    masked_predictions = predictions[mask == 1]
    masked_ground_truth = ground_truth[mask == 1]
    
    # Compute RMSE
    mse = torch.mean((masked_predictions - masked_ground_truth) ** 2)
    rmse = torch.sqrt(mse)
    
    # Compute range of ground truth values
    data_range = ground_truth.max() - ground_truth.min()
    
    # Normalize RMSE
    nrmse = rmse / data_range
    return nrmse.item()

def evaluate_model_with_nrmse(model, test_data, column_indices, missing_fraction=0.2):
    """
    Evaluates the model and computes NRMSE for masked positions.

    Args:
        model (nn.Module): Trained model.
        test_data (Tensor): Test dataset (shape: [batch_size, num_features]).
        column_indices (Tensor): Column indices for embedding (shape: [num_features]).
        missing_fraction (float): Fraction of values to mask.

    Returns:
        float: NRMSE value for the test dataset.
    """
    model.eval()
    
    # Generate masks and replace masked values with 0
    mask = create_missing_mask(test_data, missing_fraction)
    input_with_mask = test_data.clone()
    input_with_mask[mask == 1] = 0  # Replace masked positions with 0
    
    # Forward Pass
    with torch.no_grad():
        predictions = model(input_with_mask, column_indices)
    
    # Compute NRMSE
    nrmse = compute_nrmse(predictions, test_data, mask)
    print(f"NRMSE: {nrmse:.4f}")
    return nrmse

# Evaluate the Model with NRMSE
nrmse = evaluate_model_with_nrmse(model, test_tensor, column_indices)


NRMSE: 0.0254


In [8]:
from pyampute.ampute import MultivariateAmputation
import pandas as pd
import numpy as np
import torch

# Step 1: Create MCAR Dataset with pyampute
def create_mcar_dataset(data, missing_fraction=0.1):
    """
    Introduces MCAR missingness into the dataset and ensures mask matches the data.

    Args:
        data (numpy.ndarray or pd.DataFrame): Original dataset (numerical).
        missing_fraction (float): Fraction of missing values to introduce.

    Returns:
        Tensor: Data with missing values replaced by 0.0.
        Tensor: Binary mask where 1 = missing, 0 = observed.
    """
    # Convert data to DataFrame for pyampute
    if isinstance(data, np.ndarray):
        data = pd.DataFrame(data)
    
    # Apply MCAR missingness using pyampute
    amputer = MultivariateAmputation(prop=missing_fraction)
    amputed_data = amputer.fit_transform(data)
    
    # Create mask: 1 for missing values, 0 for observed
    mask = pd.isna(amputed_data).astype(int).to_numpy()  # Ensure correct shape
    amputed_data = np.nan_to_num(amputed_data, nan=0.0)  # Replace NaN with 0.0
    
    # Convert back to torch tensors
    amputed_data = torch.tensor(amputed_data, dtype=torch.float32).to(device)
    mask = torch.tensor(mask, dtype=torch.int32).to(device)
    
    # Verify shapes
    assert amputed_data.shape == mask.shape, "Data and mask shapes do not match!"
    
    return amputed_data, mask

In [9]:
# Step 2: Compute NRMSE
def compute_nrmse(predictions, ground_truth, mask):
    """
    Computes the Normalized Root Mean Squared Error (NRMSE) for imputed values.

    Args:
        predictions (Tensor): Model predictions (shape: [batch_size, num_features]).
        ground_truth (Tensor): Original data (shape: [batch_size, num_features]).
        mask (Tensor): Binary mask (1 = missing, 0 = observed).

    Returns:
        float: NRMSE value for the masked positions.
    """
    # Extract only masked values
    masked_predictions = predictions[mask == 1]
    masked_ground_truth = ground_truth[mask == 1]
    
    # Compute RMSE
    mse = torch.mean((masked_predictions - masked_ground_truth) ** 2)
    rmse = torch.sqrt(mse)
    
    # Compute range of ground truth values
    data_range = ground_truth.max() - ground_truth.min()
    
    # Normalize RMSE
    nrmse = rmse / data_range
    return nrmse.item()

In [10]:
# Step 3: Evaluate Model
def evaluate_model_with_mcar(model, original_data, column_indices, missing_fraction=0.1):
    """
    Evaluates the model using an MCAR dataset and computes NRMSE.

    Args:
        model (nn.Module): Trained model.
        original_data (Tensor): Original dataset (shape: [batch_size, num_features]).
        column_indices (Tensor): Column indices for embedding (shape: [num_features]).
        missing_fraction (float): Fraction of missing values to introduce.

    Returns:
        float: NRMSE value for the MCAR dataset.
    """
    model.eval()
    
    # Generate MCAR dataset
    amputed_data, mask = create_mcar_dataset(original_data.cpu().numpy(), missing_fraction)
    
    # Forward pass with amputed data
    with torch.no_grad():
        predictions = model(amputed_data, column_indices)
    
    # Compute NRMSE
    nrmse = compute_nrmse(predictions, original_data, mask)
    print(f"NRMSE on MCAR Data: {nrmse:.4f}")
    return nrmse

In [11]:
# Step 4: Evaluate
nrmse_mcar = evaluate_model_with_mcar(model, test_tensor, column_indices, missing_fraction=0.1)

2025-01-27 03:51:19,317 [WARNING] Failed to load lookup table for a prespecified score to probability function. It is possible data\shift_lookup.csv is missing, in the wrong location, or corrupted. Try rerunning scripts/generate_shift_lookup_table.py to regenerate the lookup table.
2025-01-27 03:51:19,320 [WARNING] Binary variables (at indices [ 3  4  7 12 24]) are indicated to be used in amputation (they are weighted and will be used to calculate the weighted sum score under MAR, MNAR, or MAR+MNAR). This can result in a subset with candidates that all have the same (or almost the same) weighted sum scores. 


NRMSE on MCAR Data: 0.0284


In [12]:
# Path to save the model
model_path = "tabular_transformer_model.pth"

# Save the model's state dictionary (recommended)
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to tabular_transformer_model.pth
